In [15]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [1]:
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }
df = pd.read_csv("../output/data/train_top_10_store_10_item.csv", dtype=dtype_dict)

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_5191/2375169706.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../output/data/train_top_10_store_10_item.csv", dtype=dtype_dict)


In [2]:

df["date"] = pd.to_datetime(df["date"])
df["unit_sales"] = df["unit_sales"].fillna(0)
df['unit_sales'] = df['unit_sales'].clip(lower=0)

In [3]:
df["date"].unique()

<DatetimeArray>
['2013-01-07 00:00:00', '2013-01-08 00:00:00', '2013-01-09 00:00:00',
 '2013-01-10 00:00:00', '2013-01-11 00:00:00', '2013-01-12 00:00:00',
 '2013-01-13 00:00:00', '2013-01-14 00:00:00', '2013-01-15 00:00:00',
 '2013-01-16 00:00:00',
 ...
 '2017-08-06 00:00:00', '2017-08-07 00:00:00', '2017-08-08 00:00:00',
 '2017-08-09 00:00:00', '2017-08-10 00:00:00', '2017-08-11 00:00:00',
 '2017-08-12 00:00:00', '2017-08-13 00:00:00', '2017-08-14 00:00:00',
 '2017-08-15 00:00:00']
Length: 1674, dtype: datetime64[ns]

In [4]:
df.describe()

,date,store_nbr,item_nbr,unit_sales
count,167304,167304.000000,1.673040e+05,167304.000000
mean,2015-04-27 04:57:39.015923200,39.099155,4.964799e+05,77.878761
min,2013-01-07 00:00:00,3.000000,1.147900e+05,0.000000
25%,2014-03-03 00:00:00,44.000000,3.143840e+05,29.423250
50%,2015-04-28 00:00:00,47.000000,5.023310e+05,53.000000
75%,2016-06-21 00:00:00,49.000000,5.828640e+05,91.321499
max,2017-08-15 00:00:00,51.000000,1.047679e+06,4635.000000
std,NaN,16.962276,2.374778e+05,88.393265


In [5]:
def generate_nonoverlap_window_features(
    df: pd.DataFrame,
    window_size: int = 5
) -> pd.DataFrame:
    """
    Splits the dates in train_df into non-overlapping windows of length `window_size`,
    then for each (store_nbr, item_nbr) within each window computes:
      - total sales on each day
      - median sales per store on each day
      - median sales per item on each day

    Returns a DataFrame with columns:
      - id = '{store}_{item}_{window_start:%Y-%m-%d}'
      - sales_day_1 ... sales_day_{window_size}
      - store_med_day_1 ... store_med_day_{window_size}
      - item_med_day_1 ... item_med_day_{window_size}'
    """
    # 1) Ensure datetime
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # 2) Build non-overlapping windows
    unique_dates = df["date"].sort_values().unique()
    chunked_windows = [
        unique_dates[i : i + window_size]
        for i in range(0, len(unique_dates), window_size)
        if len(unique_dates[i : i + window_size]) == window_size
    ]

    records = []
    for window_dates in chunked_windows:
        window_start = pd.to_datetime(window_dates[0])
        window_str   = window_start.strftime("%Y-%m-%d")

        # subset to this window
        w_df = df[df["date"].isin(window_dates)]

        # precompute medians & sums
        store_med = (
            w_df.groupby(["store_nbr","date"])["unit_sales"]
                .median()
                .unstack(fill_value=0)
        )
        item_med = (
            w_df.groupby(["item_nbr","date"])["unit_sales"]
                .median()
                .unstack(fill_value=0)
        )
        sales = (
            w_df.groupby(["store_nbr","item_nbr","date"])["unit_sales"]
                .sum()
                .unstack(fill_value=0)
        )

        for (store, item), sales_vals in sales.iterrows():
            row = {"id": f"{store}_{item}_{window_str}"}
            
            # sales_day_i
            for i, d in enumerate(window_dates, start=1):
                row[f"sales_day_{i}"] = sales_vals.get(d, 0)

            # store_med_day_i
            if store in store_med.index:
                sm = store_med.loc[store]
            else:
                sm = pd.Series(0, index=window_dates)
            for i, d in enumerate(window_dates, start=1):
                row[f"store_med_day_{i}"] = sm.get(d, 0)

            # item_med_day_i
            if item in item_med.index:
                im = item_med.loc[item]
            else:
                im = pd.Series(0, index=window_dates)
            for i, d in enumerate(window_dates, start=1):
                row[f"item_med_day_{i}"] = im.get(d, 0)

            records.append(row)

    return pd.DataFrame.from_records(records)


In [6]:
non_overlap_df = generate_nonoverlap_window_features(df, window_size=7)

In [7]:
non_overlap_df

,id,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,store_med_day_1,store_med_day_2,...,store_med_day_5,store_med_day_6,store_med_day_7,item_med_day_1,item_med_day_2,item_med_day_3,item_med_day_4,item_med_day_5,item_med_day_6,item_med_day_7
0,3_114790_2013-01-07,28.000000,11.000000,13.000,16.000000,15.000,20.000,28.000000,93.947998,51.837997,...,70.1605,77.712502,94.5,25.000000,17.500000,16.500000,15.500000,21.500000,30.500000,41.500000
1,3_305229_2013-01-07,89.000000,58.000000,81.000,49.000000,111.000,85.000,80.000000,93.947998,51.837997,...,70.1605,77.712502,94.5,36.000000,36.500000,37.500000,29.500000,37.500000,47.500000,56.500000
2,3_314384_2013-01-07,106.000000,68.000000,84.000,55.000000,72.000,90.000,114.000000,93.947998,51.837997,...,70.1605,77.712502,94.5,86.500000,63.500000,61.000000,53.000000,46.500000,94.500000,127.000000
3,3_364606_2013-01-07,125.000000,105.000000,107.000,74.000000,86.000,131.000,126.000000,93.947998,51.837997,...,70.1605,77.712502,94.5,118.000000,95.000000,94.500000,72.000000,79.500000,133.000000,158.000000
4,3_502331_2013-01-07,96.000000,59.000000,75.000,56.000000,68.000,141.000,109.000000,93.947998,51.837997,...,70.1605,77.712502,94.5,90.500000,66.000000,67.000000,48.500000,55.500000,99.500000,144.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23894,51_567623_2017-08-08,16.000000,16.000000,13.000,26.000000,15.000,23.000,13.000000,24.947500,26.396500,...,23.0000,30.696499,28.5,28.500000,21.000000,15.500000,24.500000,25.000000,38.000000,25.500000
23895,51_581078_2017-08-08,18.000000,29.000000,18.000,30.000000,26.000,31.000,17.000000,24.947500,26.396500,...,23.0000,30.696499,28.5,19.000000,23.000000,21.500000,28.000000,24.000000,32.500000,18.500000
23896,51_582864_2017-08-08,27.895000,25.792999,43.263,92.058998,15.740,30.393,47.096001,24.947500,26.396500,...,23.0000,30.696499,28.5,56.567001,57.040001,65.413498,106.737999,53.857498,64.465500,63.283501
23897,51_584028_2017-08-08,39.585999,11.647000,26.767,33.845001,14.853,9.198,3.526000,24.947500,26.396500,...,23.0000,30.696499,28.5,74.237000,72.396500,80.995499,106.775497,80.034500,104.276001,107.926498


In [8]:

# 1) Split into exactly 3 pieces at the first two underscores:
parts = non_overlap_df['id'].str.split(pat='_', n=2, expand=True)

# 2) Build your new columns
non_overlap_df['store_item'] = parts[0] + '_' + parts[1]
non_overlap_df['date']       = pd.to_datetime(parts[2])

# 3) Reorder so date, store_item come first
cols = ['date', 'store_item'] + [c for c in non_overlap_df.columns 
                                 if c not in ('date','store_item')]
non_overlap_df = non_overlap_df[cols]

non_overlap_df.head()

,date,store_item,id,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,...,store_med_day_5,store_med_day_6,store_med_day_7,item_med_day_1,item_med_day_2,item_med_day_3,item_med_day_4,item_med_day_5,item_med_day_6,item_med_day_7
0,2013-01-07,3_114790,3_114790_2013-01-07,28.0,11.0,13.0,16.0,15.0,20.0,28.0,...,70.1605,77.712502,94.5,25.0,17.5,16.5,15.5,21.5,30.5,41.5
1,2013-01-07,3_305229,3_305229_2013-01-07,89.0,58.0,81.0,49.0,111.0,85.0,80.0,...,70.1605,77.712502,94.5,36.0,36.5,37.5,29.5,37.5,47.5,56.5
2,2013-01-07,3_314384,3_314384_2013-01-07,106.0,68.0,84.0,55.0,72.0,90.0,114.0,...,70.1605,77.712502,94.5,86.5,63.5,61.0,53.0,46.5,94.5,127.0
3,2013-01-07,3_364606,3_364606_2013-01-07,125.0,105.0,107.0,74.0,86.0,131.0,126.0,...,70.1605,77.712502,94.5,118.0,95.0,94.5,72.0,79.5,133.0,158.0
4,2013-01-07,3_502331,3_502331_2013-01-07,96.0,59.0,75.0,56.0,68.0,141.0,109.0,...,70.1605,77.712502,94.5,90.5,66.0,67.0,48.5,55.5,99.5,144.5


In [9]:
non_overlap_df.columns

Index(['date', 'store_item', 'id', 'sales_day_1', 'sales_day_2', 'sales_day_3',
       'sales_day_4', 'sales_day_5', 'sales_day_6', 'sales_day_7',
       'store_med_day_1', 'store_med_day_2', 'store_med_day_3',
       'store_med_day_4', 'store_med_day_5', 'store_med_day_6',
       'store_med_day_7', 'item_med_day_1', 'item_med_day_2', 'item_med_day_3',
       'item_med_day_4', 'item_med_day_5', 'item_med_day_6', 'item_med_day_7'],
      dtype='object')

In [18]:
non_overlap_df.drop(columns=['id'], inplace=True)

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_5191/1934350804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_overlap_df.drop(columns=['id'], inplace=True)


In [19]:
#non_overlap_df.drop(columns="id", inplace=True)
non_overlap_df.head()

,date,store_item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,store_med_day_1,...,store_med_day_5,store_med_day_6,store_med_day_7,item_med_day_1,item_med_day_2,item_med_day_3,item_med_day_4,item_med_day_5,item_med_day_6,item_med_day_7
0,2013-01-07,3_114790,28.0,11.0,13.0,16.0,15.0,20.0,28.0,93.947998,...,70.1605,77.712502,94.5,25.0,17.5,16.5,15.5,21.5,30.5,41.5
1,2013-01-07,3_305229,89.0,58.0,81.0,49.0,111.0,85.0,80.0,93.947998,...,70.1605,77.712502,94.5,36.0,36.5,37.5,29.5,37.5,47.5,56.5
2,2013-01-07,3_314384,106.0,68.0,84.0,55.0,72.0,90.0,114.0,93.947998,...,70.1605,77.712502,94.5,86.5,63.5,61.0,53.0,46.5,94.5,127.0
3,2013-01-07,3_364606,125.0,105.0,107.0,74.0,86.0,131.0,126.0,93.947998,...,70.1605,77.712502,94.5,118.0,95.0,94.5,72.0,79.5,133.0,158.0
4,2013-01-07,3_502331,96.0,59.0,75.0,56.0,68.0,141.0,109.0,93.947998,...,70.1605,77.712502,94.5,90.5,66.0,67.0,48.5,55.5,99.5,144.5


In [20]:
non_overlap_df.to_pickle("../output/data/train_nonoverlap_top_10_store_item.pkl")

In [21]:
non_overlap_df.columns[2:]

Index(['sales_day_1', 'sales_day_2', 'sales_day_3', 'sales_day_4',
       'sales_day_5', 'sales_day_6', 'sales_day_7', 'store_med_day_1',
       'store_med_day_2', 'store_med_day_3', 'store_med_day_4',
       'store_med_day_5', 'store_med_day_6', 'store_med_day_7',
       'item_med_day_1', 'item_med_day_2', 'item_med_day_3', 'item_med_day_4',
       'item_med_day_5', 'item_med_day_6', 'item_med_day_7'],
      dtype='object')

In [22]:
# --- Apply MinMax Scaling ---
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(non_overlap_df.drop(columns=["date","store_item"]))
scaled_df = pd.DataFrame(scaled_values, columns=non_overlap_df.columns[2:])
scaled_df.insert(0, "date", non_overlap_df["date"])
scaled_df.insert(1, "store_item", non_overlap_df["store_item"])
scaled_df.head()


,date,store_item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,store_med_day_1,...,store_med_day_5,store_med_day_6,store_med_day_7,item_med_day_1,item_med_day_2,item_med_day_3,item_med_day_4,item_med_day_5,item_med_day_6,item_med_day_7
0,2013-01-07,3_114790,0.022222,0.002373,0.003231,0.006214,0.010830,0.017841,0.022951,0.480343,...,0.293247,0.349928,0.410431,0.037759,0.021672,0.019727,0.018939,0.029139,0.058366,0.080148
1,2013-01-07,3_305229,0.070635,0.012513,0.020134,0.019029,0.080144,0.075825,0.065574,0.480343,...,0.293247,0.349928,0.410431,0.064555,0.080495,0.083460,0.071970,0.071523,0.102464,0.117139
2,2013-01-07,3_314384,0.084127,0.014671,0.020880,0.021359,0.051986,0.080285,0.093443,0.480343,...,0.293247,0.349928,0.410431,0.187576,0.164087,0.154780,0.160985,0.095364,0.224384,0.290999
3,2013-01-07,3_364606,0.099206,0.022654,0.026597,0.028738,0.062094,0.116860,0.103279,0.480343,...,0.293247,0.349928,0.410431,0.264312,0.261610,0.256449,0.232955,0.182781,0.324254,0.367448
4,2013-01-07,3_502331,0.076190,0.012729,0.018643,0.021748,0.049097,0.125781,0.089344,0.480343,...,0.293247,0.349928,0.410431,0.197320,0.171827,0.172989,0.143939,0.119205,0.237354,0.334155


In [ ]:
 # Save to dataframe
scaled_df.to_pickle("../output/data/scaled_train_nonoverlap_top_10_store_item.pkl")

In [24]:
# 1) Define all 21 “today” feature cols
feat_cols = (
    [f"sales_day_{i}"     for i in range(1, 8)]  # 7 cols
  + [f"store_med_day_{i}" for i in range(1, 8)]  # 7 cols
  + [f"item_med_day_{i}"  for i in range(1, 8)]  # 7 cols
)

# 2) Create “next‑day” labels for the same 21 cols
df = scaled_df.sort_values(['store_item','date']).copy()
for col in feat_cols:
    df[f"y_{col}"] = df.groupby('store_item')[col].shift(-1)

df.head()

,date,store_item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,store_med_day_1,...,y_store_med_day_5,y_store_med_day_6,y_store_med_day_7,y_item_med_day_1,y_item_med_day_2,y_item_med_day_3,y_item_med_day_4,y_item_med_day_5,y_item_med_day_6,y_item_med_day_7
9,2013-01-07,3_1047679,0.458730,0.073355,0.098931,0.201942,0.392058,0.811775,0.642623,0.480343,...,0.248855,0.293631,0.373440,0.655298,0.424149,0.176024,0.202652,0.437086,0.675746,0.711467
109,2013-01-14,3_1047679,0.370635,0.099245,0.067611,0.029903,0.287365,0.652988,0.554918,0.240328,...,0.233335,0.264983,0.296819,0.434836,0.191950,0.206373,0.272727,0.214570,0.282750,0.514180
209,2013-01-21,3_1047679,0.461111,0.074649,0.086751,0.156117,0.161733,0.463872,0.388525,0.361615,...,0.379813,0.355046,0.342084,0.080390,0.199690,0.147193,0.196970,0.209272,0.363165,0.329223
309,2013-01-28,3_1047679,0.199206,0.065156,0.071588,0.128932,0.391336,0.333631,0.526230,0.304991,...,0.225370,0.306709,0.184635,0.225335,0.202786,0.209408,0.225379,0.182781,0.225681,0.124538
409,2013-02-04,3_1047679,0.247619,0.076160,0.065374,0.104854,0.340072,0.325602,0.200820,0.315430,...,0.309908,0.354130,0.308045,0.152253,0.241486,0.210926,0.170455,0.135099,0.172503,0.186190


In [ ]:
# 1) extract item_nbr (the part after the underscore)
df['item_nbr'] = df['store_item'].str.split('_', expand=True)[1].astype(int)

# 2) move it to right after 'store_item'
pos = df.columns.get_loc('store_item') + 1
df.insert(pos, 'item_nbr', df.pop('item_nbr'))
df.head()

In [25]:
df.query('store_item == "3_1047679"')


,date,store_item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,sales_day_7,store_med_day_1,...,y_store_med_day_5,y_store_med_day_6,y_store_med_day_7,y_item_med_day_1,y_item_med_day_2,y_item_med_day_3,y_item_med_day_4,y_item_med_day_5,y_item_med_day_6,y_item_med_day_7
9,2013-01-07,3_1047679,0.458730,0.073355,0.098931,0.201942,0.392058,0.811775,0.642623,0.480343,...,0.248855,0.293631,0.373440,0.655298,0.424149,0.176024,0.202652,0.437086,0.675746,0.711467
109,2013-01-14,3_1047679,0.370635,0.099245,0.067611,0.029903,0.287365,0.652988,0.554918,0.240328,...,0.233335,0.264983,0.296819,0.434836,0.191950,0.206373,0.272727,0.214570,0.282750,0.514180
209,2013-01-21,3_1047679,0.461111,0.074649,0.086751,0.156117,0.161733,0.463872,0.388525,0.361615,...,0.379813,0.355046,0.342084,0.080390,0.199690,0.147193,0.196970,0.209272,0.363165,0.329223
309,2013-01-28,3_1047679,0.199206,0.065156,0.071588,0.128932,0.391336,0.333631,0.526230,0.304991,...,0.225370,0.306709,0.184635,0.225335,0.202786,0.209408,0.225379,0.182781,0.225681,0.124538
409,2013-02-04,3_1047679,0.247619,0.076160,0.065374,0.104854,0.340072,0.325602,0.200820,0.315430,...,0.309908,0.354130,0.308045,0.152253,0.241486,0.210926,0.170455,0.135099,0.172503,0.186190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23408,2017-07-11,3_1047679,0.257937,0.057174,0.067860,0.117670,0.267870,0.435326,0.264754,0.210064,...,0.357892,0.246844,0.215483,0.250914,0.294118,0.256449,0.337121,0.425166,0.485084,0.204686
23508,2017-07-18,3_1047679,0.378571,0.089752,0.071588,0.135534,0.288809,0.529884,0.231967,0.228311,...,0.223454,0.218043,0.236825,0.222899,0.280186,0.236722,0.350379,0.425166,0.404669,0.236745
23608,2017-07-25,3_1047679,0.228571,0.072708,0.061894,0.128932,0.311191,0.331847,0.331967,0.171233,...,0.217243,0.326331,0.205293,0.489647,0.679567,0.511381,0.784091,0.529801,0.596628,0.300863
23708,2017-08-01,3_1047679,0.234127,0.091694,0.128262,0.181359,0.319856,0.314005,0.209836,0.278098,...,0.205472,0.204663,0.192660,0.181486,0.354489,0.330804,0.583333,0.369536,0.324254,0.271270


In [28]:
print("Unique (store, item) pairs:", df["store_item"].nunique())


Unique (store, item) pairs: 100


In [30]:
df.to_pickle("../output/data/scaled_train_nonoverlap_top_10_store_item_X_y.pkl")